<details><summary> </summary>

# Skip notebook test

</details>

In [ ]:
import os
import pandas as pd
import notebook_utils.notebook_helpers as utils
from cuopt_thin_client import CuOptServiceClient

# Re-optimization

## Capacitated Pickup and Delivery Problem with Time Windows


In a factory set-up, routing needs to be optimized as and when a new set of tasks/requests are dispatched, this would help re-route robots to any other new task which got dispatched on the floor. This would require knowledge of all the robot's statuses like current location, tasks yet to done, and many other factors. 

In this scenario, we have 4 robots which are in a factory and serves 5 locations/nodes in the factory. Factory would be receiving new tasks of pickup and delivery for 3 different time stamps, and this would trigger rerouting to efficiently complete all the tasks.

### Problem Details:

- 5 locations
- 4 vehicles/robots
    - capacity: [3, 3, 3, 3]
    - vehicle start location : [0, 0, 0, 0]
    - vehicle end location: [0, 0, 0, 0]
    
At each time stamp, a set of tasks gets added:

- Time Stamp: 0 time unit
    - tasks :  [1,  2, 3,  4, 3,  2, 1,  3]
- Time Stamp: 4 time units
    - tasks : [4, 2, 3, 1]
- Time Stamp: 6 time units
    - tasks : [2, 3]

### Assumptions
- Fleet size and capacity would remain same
- Vehicles en route to service a particular task will finish that task first
- The original optimized plan is going according to the plan, so we can determine the finished tasks just based on the time stamp
- The problem is pickup and delivery only
- There is only one capacity demand dimension

In [ ]:
cuopt_client_id = os.environ["CUOPT_CLIENT_ID"]
cuopt_client_secret = os.environ["CUOPT_CLIENT_SECRET"]


cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

In [ ]:
 # Processes input data, communicates data to cuOpt server and returns Optimized routes
def get_optimized_route(data):
    
    cuopt_problem_data = {"task_data":{},"fleet_data":{}}
    

    # Set cost matrix
    if "cost_matrices" in data:
        cuopt_problem_data["cost_matrix_data"] = {
        "cost_matrix": data["cost_matrices"]
    }
        
    
    # Set/Update Transit time matrix    
    if "transit_time_matrices" in data:
        cuopt_problem_data["travel_time_matrix_data"] = {
        "cost_matrix": data["transit_time_matrices"]
    }
            
    
    # Set/Update Task data
    if "task_locations" in data:
        
        cuopt_problem_data["task_data"]["task_locations"] = data["task_locations"]
    if "pickup_indices" in data and "delivery_indices" in data:
        cuopt_problem_data["task_data"]["pickup_and_delivery_pairs"] = [
            [pickup_idx, delivery_idx] for pickup_idx, delivery_idx in zip(
                data["pickup_indices"], data["delivery_indices"]
            )
        ]
    elif "pickup_indices" in data or "delivery_indices" in data:
        raise ValueError("Pick indices or Delivery indices are missing, both should be provided")
    
    if "task_earliest_time" in data and "task_latest_time" in data and "task_service_time" in data:
        cuopt_problem_data["task_data"]["task_time_windows"] = [
            [earliest, latest] for earliest, latest in zip(data["task_earliest_time"], data["task_latest_time"])
        ]
        cuopt_problem_data["task_data"]["service_times"] =  data["task_service_time"]
    elif "task_earliest_time" in data or "task_latest_time" in data or "task_service_time" in data:
        raise ValueError("Earliest, Latest and Service time should be provided, one or more are missing")
        
    if "demand" in data:
        cuopt_problem_data["task_data"]["demand"] = data["demand"]
    
    if "order_vehicle_match" in data:
        cuopt_problem_data["task_data"]["order_vehicle_match"] = data["order_vehicle_match"]

        
    # Set/Update Fleet data
    if "vehicle_locations" in data:
        cuopt_problem_data["fleet_data"]["vehicle_locations"] = data["vehicle_locations"]
        
    if "capacity" in data:
        cuopt_problem_data["fleet_data"]["capacities"] = data["capacity"]
        
    if "vehicle_earliest" in data and "vehicle_latest" in data:
        cuopt_problem_data["fleet_data"]["vehicle_time_windows"] = [
            [earliest, latest] for earliest, latest in zip(data["vehicle_earliest"], data["vehicle_latest"])
        ]
    elif "vehicle_earliest" in data or "vehicle_latest" in data:
        raise ValueError("vehicle_earliest and vehicle_latest both should be provided, one of them is missing")
       
    
    # Set Solver settings
    cuopt_problem_data["solver_config"] = {
        "time_limit": 5
    }  
    
    # Get optimized route
    solver_response = cuopt_service_client.get_optimized_routes(
        cuopt_problem_data
    )
    
    return solver_response["response"]["solver_response"]

 ### Following function accumulates status of tasks and vehicles
 
 - Collect status of tasks [Completed, Picked-up but not yet Delivered, Yet to Pick-up]
 - Along with that update vehicle locations, vehicle earliest times at a given time.
 - It is also required to keep tabs on tasks which have been picked up by particular vehicle, to ensure that
   that task is fulfilled by the same vehicle.

In [ ]:
def collect_current_status(previous_data, optimized_route_data, reroute_from_time):
    
    transit_times = previous_data["transit_time_matrices"][0]
    task_locations = previous_data["task_locations"]
    vehicle_start_locations = [val[0] for val in previous_data["vehicle_locations"]]
    vehicle_return_locations = [val[1] for val in previous_data["vehicle_locations"]]

    # Create a mapping between pickup and delivery
    pickup_of_delivery = {
        previous_data["delivery_indices"][i]: previous_data["pickup_indices"][i]
        for i in range(len(previous_data["pickup_indices"]))
    }
    
    # Update vehicle earliest if needs to be changed to current time
    vehicle_earliest = [max(earliest, reroute_from_time) for earliest in previous_data["vehicle_earliest"]]
    
    # Collect completed and partial set of tasks, so we can add partialy completed tasks back
    completed_tasks = []
    picked_up_but_not_delivered = {}
    picked_up_task_to_vehicle = {}
    
    for veh_id, veh_data in optimized_route_data["vehicle_data"].items():
        task_ids = [int(t_id) for t_id in veh_data['task_id'][1:-1]]  # Ignore Depot
        arrival_stamps = veh_data['arrival_stamp'][1:-1]  # Ignore Depot
        route_len = len(veh_data['route'])
        task_len = len(task_ids)
        vehicle_id = int(veh_id)
    
        # In this case, all the tasks are already completed, or waiting on last task service time
        if arrival_stamps[-1] <= reroute_from_time:
            intra_task_id = task_len
        else:
            try:
                # Look for a task that is yet to be completed
                intra_task_id, time = next(
                    (i, el)
                    for i, el in enumerate(arrival_stamps)
                    if el > reroute_from_time
                )
            except StopIteration:
                # In case none of the tasks are completed
                intra_task_id = 0
                time = max(vehicle_earliest[vehicle_id], reroute_from_time)

        # All the tasks are completed and vehicle is on the way to return location or already reached

        picked_up_but_not_delivered[vehicle_id] = []
            
        # There are tasks that are still pending
        if intra_task_id < task_len:

            last_task = int(task_ids[intra_task_id])
            
            # Update vehicle start location
            vehicle_start_locations[int(vehicle_id)] = task_locations[last_task]
            
            # Update vehicle earliest
            vehicle_earliest[int(vehicle_id)] = min(
                max(time, reroute_from_time), previous_data["vehicle_latest"][vehicle_id]
            )
            
            for j in range(0, intra_task_id):
                task = task_ids[j]
                if task != 'Depot':
                    task = int(task)
                    if task in previous_data["pickup_indices"]:
                        picked_up_but_not_delivered[vehicle_id].append(task)
                        picked_up_task_to_vehicle[task] = vehicle_id
                    else:
                        # Moves any delivered pick-up tasks to completed.
                        corresponding_pickup = pickup_of_delivery[task]
                        picked_up_but_not_delivered[vehicle_id].remove(
                                corresponding_pickup
                        )
                        completed_tasks.append(corresponding_pickup)
                        completed_tasks.append(task)
                        picked_up_task_to_vehicle.pop(corresponding_pickup)
        else:
            completed_tasks.extend(task_ids)
            # In this case vehicle is at last location about to finish the task,
            # so vehicle start location would last task location and accordingly the earliest vehicle time as well
            if (arrival_stamps[-1] == reroute_from_time) and (arrival_stamps[-1]+previous_data["task_service_time"][task_ids[-1]] >= reroute_from_time):
                vehicle_start_locations[vehicle_id] = task_locations[task_ids[-1]]
                vehicle_earliest[vehicle_id] = arrival_stamps[-1] + previous_data["task_service_time"][task_ids[-1]] 
            else:
                # In this case vehicle completed last task and may be enroute to vehicle return location or might have reached.
                end_time = (
                    arrival_stamps[-1] + previous_data["task_service_time"][task_ids[-1]] + transit_times[task_locations[task_ids[-1]]][vehicle_return_locations[vehicle_id]]
                )
                time = max(end_time, reroute_from_time)
                print("For vehicle ID updating", vehicle_id)
                vehicle_start_locations[vehicle_id] = vehicle_return_locations[vehicle_id]
                vehicle_earliest[vehicle_id] = min(time, previous_data["vehicle_earliest"][vehicle_id])
                
    return (
        vehicle_earliest, vehicle_start_locations,
        vehicle_return_locations, completed_tasks,
        picked_up_but_not_delivered, picked_up_task_to_vehicle)

### Redefine old data at current time
- Unfinished task data gets moved up the task locations
- This changes task id which affects pick-up and delivery indices, so they get remapped
- Similarly update the earliest time appropriately, since this might change with current time

In [ ]:
def redefine_old_data_to_current_time(
    previous_data, vehicle_earliest, vehicle_start_locations,
    completed_tasks, picked_up_task_to_vehicle, reroute_from_time
):
    data = {
        "task_locations" : [],
        "pickup_indices" : [],
        "delivery_indices" : [],
        "task_earliest_time" : [],
        "task_latest_time" : [],
        "task_service_time" : [],
        # assume that fleet is not changing
        "vehicle_capacity" : previous_data["capacity"],
        "task_demand" : [[]*len(previous_data["capacity"])],
    }
   
    # If there are unfinished tasks (picked up but not delivered yet) at
    # the time of re-routing, create new pickup tasks from the vehicle
    # locations at time zero (i.e. re-routing start time)
    
    cnt = 0
    # Mapping new task Id to old task Id and old task Id to new task Id
    new_task_to_old_task = {}
    old_task_to_new_task = {}
    
    ntasks = len(previous_data["task_locations"])
    
    for task_id in range(0, ntasks):
        if completed_tasks.count(task_id) == 0:
            new_task_to_old_task[cnt] = task_id
            old_task_to_new_task[task_id] = cnt
            cnt = cnt + 1
    
    # Convert pickup and delivery indices to new numbering
    for i in range(len(previous_data["pickup_indices"])):
        pickup = previous_data["pickup_indices"][i]
        delivery = previous_data["delivery_indices"][i]
        if pickup not in completed_tasks:
            new_pickup = old_task_to_new_task[pickup]
            new_delivery = old_task_to_new_task[delivery]
            data["pickup_indices"].append(new_pickup)
            data["delivery_indices"].append(new_delivery)
            
    # Convert task to vehicle to new numbering
    new_picked_up_task_to_vehicle = [
        {
            "order_id" :old_task_to_new_task[pickup],
            "vehicle_ids" :[vehicle]
        }
        for pickup, vehicle in picked_up_task_to_vehicle.items()
    ]
    
    new_picked_up_task_list = [val["order_id"] for val in new_picked_up_task_to_vehicle]
    
    # Extract the task info of incomplete tasks
    for i in range(0, len(new_task_to_old_task)):
        old_id = new_task_to_old_task[i]
        is_pickup_task = i in data["pickup_indices"]

        # If this task is already picked up, make sure that in the new problem
        # it is picked up at time zero
        if is_pickup_task and i in new_picked_up_task_list:
            vehicle = picked_up_task_to_vehicle[old_id]
            new_loc = vehicle_start_locations[vehicle]
            pickup_time = vehicle_earliest[vehicle]
            data["task_locations"].append(new_loc)
            data["task_earliest_time"].append(pickup_time)
            data["task_latest_time"].append(pickup_time)
            data["task_service_time"].append(0)
        else:
            data["task_locations"].append(previous_data["task_locations"][old_id])
            data["task_earliest_time"].append(max(previous_data["task_earliest_time"][old_id], reroute_from_time))
            data["task_latest_time"].append(previous_data["task_latest_time"][old_id])
            data["task_service_time"].append(previous_data["task_service_time"][old_id])

        for idx, demands in enumerate(previous_data["demand"]):
            # assume that fleet is not changing
            demand = demands[old_id]
            data["task_demand"][idx].append(demand)
            
    data["order_vehicle_match"] = new_picked_up_task_to_vehicle
            
    return data, new_task_to_old_task, old_task_to_new_task

### Reconstructs data adding pending tasks with new tasks taking current env status

In [ ]:
def reconstruct_task_data(
    previous_data, optimized_route_data, reroute_from_time, new_task_data,
):  
    # Approach:
    # ---------
    # 1. Using the optimized route and the time of re-optimization, we figure
    #    out which tasks are fulfilled (picked up and delivered), partially
    #    fulfilled (picked up but not delivered), and not initiated
    # 2. We remove the tasks that are fulfilled while keeping the tasks that
    #    are not initiated. For the partially fulfilled tasks, we create dummy
    #    pickup tasks at vehicle start locations
    
    if new_task_data is not None:
        expected_entries_in_new_task_data = [
            "task_locations",
            "task_earliest_time",
            "task_latest_time",
            "task_service_time",
            "pickup_indices",
            "delivery_indices",
            "demand",
        ]
        for entry in expected_entries_in_new_task_data:
            if entry not in new_task_data:
                raise ValueError(f"{entry} is missing in new task data")

        for entry, value in new_task_data.items():
            if entry not in expected_entries_in_new_task_data:
                raise NotImplementedError(
                    f"{entry} is not implemented for re-optimization"
                )
    
    if "cost_matrices" in new_task_data:
        if not len(new_task_data["cost_matrices"][0]) == len(previous_data["cost_matrices"][0]):
            raise ValueError("Shape of cost matrices is not matching")
    
    if "transit_time_matrices" in new_task_data:
        if not len(new_task_data["transit_time_matrices"][0]) == len(previous_data["transit_time_matrices"][0]):
            raise ValueError("Shape of transit time matrices is not matching")
                                                        
    
    vehicle_num = len(previous_data["vehicle_locations"])
    n_locations = len(previous_data["cost_matrices"][0])
    task_locations = previous_data["task_locations"]
    
    # - Collect status of tasks [Completed, Picked-up note yet Delivered, Yet to Pick-up]
    # - Along with that update vehicle locations, vehicle earliest times at given time.
    # - It is also required to keep tabs on tasks which have been picked up by particular vehicle, to ensure that
    #   that task is fulfilled by same vehicle.
    (
        vehicle_earliest, vehicle_start_locations,
        vehicle_return_locations, completed_tasks,
        picked_up_but_not_delivered, picked_up_task_to_vehicle
    ) = collect_current_status(previous_data, optimized_route_data, reroute_from_time)
    
    
    # Map old task as new tasks with given time, in this the old task gets new ids since they are moved
    # upfront in task locations removing completed tasks.
    restructured_data, new_task_to_old_task, old_task_to_new_task = redefine_old_data_to_current_time(
        previous_data, vehicle_earliest, vehicle_start_locations,
        completed_tasks, picked_up_task_to_vehicle, reroute_from_time
    )
    
    n_leftover_tasks = len(restructured_data["task_locations"])
    
    # Append new task data to pending tasks details
    
    restructured_data["task_locations"].extend(new_task_data["task_locations"])
    restructured_data["task_earliest_time"].extend(new_task_data["task_earliest_time"])
    restructured_data["task_latest_time"].extend(new_task_data["task_latest_time"])
    restructured_data["task_service_time"].extend(new_task_data["task_service_time"])

    # new task data consists of indices with respect to new task data
    adjusted_pickup_indices = [
        id + n_leftover_tasks for id in new_task_data["pickup_indices"]
    ]
    adjusted_delivery_indices = [
        id + n_leftover_tasks for id in new_task_data["delivery_indices"]
    ]
    restructured_data["pickup_indices"].extend(adjusted_pickup_indices)
    restructured_data["delivery_indices"].extend(adjusted_delivery_indices)
    
    restructured_data["task_demand"] = [
        old_demand + new_demand for old_demand, new_demand in zip(
            restructured_data["task_demand"], new_task_data["demand"]
        )
    ]
    
    restructured_data["vehicle_locations"] = [
        [start, ret] for start, ret in zip(vehicle_start_locations, vehicle_return_locations)
    ]
    
    # Form the output data
    
    reconstructed_data = {
        "cost_matrices" : (
            new_task_data["cost_matrices"] if "cost_matrices" in new_task_data else previous_data["cost_matrices"]
        ),
        "transit_time_matrices" : (
            new_task_data["transit_time_matrices"] if "transit_time_matrices" in new_task_data else previous_data["transit_time_matrices"]
        ),
        "task_locations": restructured_data["task_locations"],
        "pickup_indices": restructured_data["pickup_indices"], 
        "delivery_indices": restructured_data["delivery_indices"],
        "task_earliest_time": restructured_data["task_earliest_time"],
        "task_latest_time": restructured_data["task_latest_time"],
        "task_service_time": restructured_data["task_service_time"],
        "demand": restructured_data["task_demand"],
        "capacity": restructured_data["vehicle_capacity"],
        "vehicle_locations": restructured_data["vehicle_locations"],
        "vehicle_earliest": vehicle_earliest,
        "vehicle_latest": previous_data["vehicle_latest"],
        "order_vehicle_match": restructured_data["order_vehicle_match"]
    }
    
    return reconstructed_data, completed_tasks

## Reoptimization across different time stamps
________________________________________________


## Time Stamp : 0 time unit
- Solver gets first set of tasks to plan for their optimized routes
- Vehicles/Robots all start from their start locations

In [ ]:
# Input data - 1

input_data_1 = {}

input_data_1["cost_matrices"] = {0:[
    [0, 1, 2, 3, 4],
    [1, 0, 2, 3, 4],
    [2, 3, 0, 4, 1],
    [3, 4, 1, 0, 2],
    [4, 1, 2, 3, 0]
]}

input_data_1["transit_time_matrices"] = {0:[
    [0, 1, 1, 1, 1],
    [1, 0, 1, 1, 1],
    [1, 1, 0, 1, 1],
    [1, 1, 1, 0, 1],
    [1, 1, 1, 1, 0]
]}
       
# Task data
input_data_1["task_locations"] = [1, 2, 3, 4, 3, 2, 1, 3]
input_data_1["pickup_indices"]   = [0, 2, 4, 6]
input_data_1["delivery_indices"] = [1, 3, 5, 7]
input_data_1["task_earliest_time"] = [0, 2, 1, 2, 0, 0,  2, 1]
input_data_1["task_latest_time"]   = [5, 4, 5, 5, 7, 7, 10, 9]
input_data_1["task_service_time"]  = [1, 1, 1, 1, 1, 1,  1, 1]
input_data_1["demand"] = [[1, -1, 1, -1, 1, -1, 1, -1]]

# Vehicle data
input_data_1["vehicle_locations"] = [[0, 0]] * 4 
input_data_1["vehicle_earliest"] = [0.0, 0.0,  2.0,  2.0]
input_data_1["vehicle_latest"]   = [8.0, 8.0, 15.0, 15.0]
input_data_1["capacity"] = [[3, 3, 3, 3]]

# Solve for optimized route
resp = get_optimized_route(input_data_1)

#### Optimized route

In [ ]:
utils.print_vehicle_data(resp)

## Time Stamp: 4 time units
- 4 New tasks gets as added
- it's a pair of pick-up and delivery tasks
- As per observation of the result above, at time stamp 4, only 2 pair of pick-up and delivery is completed, 2 are still pendng

In [ ]:
# time stamnp
current_time = 4

# new set of tasks

new_task_data = {}
new_task_data["task_locations"] = [4, 2, 3, 1]
new_task_data["pickup_indices"]   = [0, 2]
new_task_data["delivery_indices"] = [1, 3]
new_task_data["task_earliest_time"] = [3, 4, 8,  9]
new_task_data["task_latest_time"] =   [8, 9, 10, 10]
new_task_data["task_service_time"] =  [1, 1, 1,  1]
new_task_data["demand"] = [[1, -1, 1, -1]]

#### Reconstruct task data adding pending tasks and new tasks

In [ ]:
updated_task_data, completed_tasks = reconstruct_task_data(input_data_1, resp, current_time, new_task_data)

In [ ]:
utils.print_data(updated_task_data, completed_tasks)

#### Analyzing updated task data

- It can be observed that now the updated task data has 4 more tasks along with 4 new tasks, which makes it 4 sets of pickup and delivery
- The vehicle earliest, vehicle start locations have been updated with current location of the robots
- The previous task which were picked-up but not yet delivered are assigned to the same vehicles which had picked them up through task to vehicle match
- All the vehicle earliest is changed to current time if they were less than that

In [ ]:
resp = get_optimized_route(updated_task_data)

#### Optimized route

In [ ]:
utils.print_vehicle_data(resp)

## Time Stamp: 6 time units
- 2 New tasks gets as added
- it's a pick-up and delivery
- As per observation of the result above, at time stamp 6, only 2 pair of pick-up and delivery is completed, 2 are still pendng and now one more pickup and delivery is added

In [ ]:
# time stamnp
current_time = 6

# new set of tasks

new_task_data = {}
new_task_data["task_locations"] = [2, 3]
new_task_data["pickup_indices"]   = [0]
new_task_data["delivery_indices"] = [1]
new_task_data["task_earliest_time"] = [10, 9]
new_task_data["task_latest_time"] =   [12, 14]
new_task_data["task_service_time"] =  [1, 1]
new_task_data["demand"] = [[1, -1]]

In [ ]:
updated_task_data, completed_tasks = reconstruct_task_data(updated_task_data, resp, current_time, new_task_data)

In [ ]:
utils.print_data(updated_task_data, completed_tasks)

#### Analyzing updated task data
- Out of 8 tasks, 4 are completed and 4 are pending and we are adding 2 more, at the end we have 3 pair of pick-up and delivery
- Vehicle start locations have changed to location 2 and 3 for vehicles 2 and 3.
- Only task 0 is bound to a vehicle sice it already picked-up before

#### Optimized route

In [ ]:
resp = get_optimized_route(updated_task_data)

In [ ]:
utils.print_vehicle_data(resp)